In [ ]:
import pandas as pd
import pingouin as pg
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_validate, StratifiedKFold, cross_val_predict, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

import optuna

In [ ]:
df_segmento = pd.read_csv('./datasets/data.csv')

## EDA

In [ ]:
df_segmento.head(10)

In [ ]:
df_segmento.info()

In [ ]:
df_segmento['atividade_economica'].unique()

In [ ]:
df_segmento['localizacao'].unique()

In [ ]:
df_segmento['segmento_de_cliente'].unique()


In [ ]:
df_segmento['inovacao'].unique()

In [ ]:
contagem_target = df_segmento.value_counts('segmento_de_cliente')
contagem_target

In [ ]:
lista_segmento = ['Starter', 'Bronze', 'Silver', 'Gold']

In [ ]:
px.bar(contagem_target, color=contagem_target.index, category_orders={'segmento_de_cliente': lista_segmento})

In [ ]:
percentual_target = contagem_target / len(df_segmento) * 100
px.bar(percentual_target, color=percentual_target.index, category_orders={'segmento_de_cliente': lista_segmento})

In [ ]:
percentual_localizacao = df_segmento.value_counts('localizacao') / len(df_segmento) * 100
px.bar(percentual_localizacao, color=percentual_localizacao.index)

In [ ]:
percentual_atividade_economica = df_segmento.value_counts('atividade_economica') / len(df_segmento) * 100
px.bar(percentual_atividade_economica, color=percentual_atividade_economica.index)

In [ ]:
percentual_inovacao = df_segmento.value_counts('inovacao') / len(df_segmento) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)

In [ ]:
crosstab_localizacao = pd.crosstab(df_segmento['localizacao'], df_segmento['segmento_de_cliente'], margins=True)[lista_segmento].reset_index()
tabela_localizacao = ff.create_table(crosstab_localizacao)

tabela_localizacao.show()

In [ ]:
crosstab_atividade = pd.crosstab(df_segmento['atividade_economica'], df_segmento['segmento_de_cliente'], margins=True)[lista_segmento].reset_index()
tabela_atividade = ff.create_table(crosstab_atividade)

tabela_atividade.show()

In [ ]:
crosstab_inovacao = pd.crosstab(df_segmento['inovacao'], df_segmento['segmento_de_cliente'], margins=True)[lista_segmento].reset_index()
tabela_inovacao = ff.create_table(crosstab_inovacao)

tabela_inovacao.show()

In [ ]:
px.histogram(df_segmento, x='idade')

In [ ]:
px.histogram(df_segmento, x='faturamento_mensal')

In [ ]:
px.box(df_segmento, x='segmento_de_cliente', y='idade', color='segmento_de_cliente', category_orders={'segmento_de_cliente': lista_segmento})

In [ ]:
px.box(df_segmento, x='segmento_de_cliente', y='faturamento_mensal', color='segmento_de_cliente', category_orders={'segmento_de_cliente': lista_segmento})

In [ ]:
valor_esperado, valor_observado, estatisticas = pg.chi2_independence(df_segmento, 'segmento_de_cliente', 'inovacao')

In [ ]:
# Valor esperado
# Frequência que seria esperado se não houvesse associação entre as variáveis
# É calculado usando a distribuição assumida no teste qui-quadrado
valor_esperado

In [ ]:
# Valor observado
# É a frequência real dos dados coletados
valor_observado

In [ ]:
# Estatisticas
estatisticas.round(5)

As variáveis Localização e Segmento de Cliente são independentes. Qui-quadrado (p-value = 0.81714)
As variáveis atividade econômica e Segmento de Cliente são independentes. Qui-quadrado (p-value = 0.35292)
As variáveis inovação e Segmento de Cliente não são independentes. Qui-quadrado (p-value = 0.0)

## Treinando o modelo

In [ ]:
X = df_segmento.drop(columns=['segmento_de_cliente'])
y = df_segmento['segmento_de_cliente']

In [ ]:
categorical_features = ['atividade_economica', 'localizacao']
categorical_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='most_frequent')),
  ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
  transformers=[
    ('cat', categorical_transformer, categorical_features)
  ]
)

dt_model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', DecisionTreeClassifier())])

### Validação Cruzada

In [ ]:
cv_folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=51)
metrics_result = cross_validate(dt_model, X, y, cv=cv_folds, scoring=['accuracy'], return_estimator=True)

In [ ]:
metrics_result

In [ ]:
metrics_result['test_accuracy'].mean()

### Metricas

In [ ]:
y_pred = cross_val_predict(dt_model, X, y, cv=cv_folds)

In [ ]:
classification_report_str = classification_report(y, y_pred)

print(f'Relatório de classificação:\n{classification_report_str}')

In [ ]:
confusion_matrix_modelo = confusion_matrix(y, y_pred, labels=lista_segmento)
disp = ConfusionMatrixDisplay(confusion_matrix_modelo, display_labels=lista_segmento)
disp.plot()

### Tuning de Hiperparâmetros

In [ ]:
# min_samples_leaf = Mínimo de instâncias requerido para formar uma folha (nó terminal)
# max_depth = Profundidade máxima da árvore

def decisiontree_optuna(trial):
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
  max_depth = trial.suggest_int('max_depth', 2, 8)

  dt_model.set_params(classifier__min_samples_leaf=min_samples_leaf)
  dt_model.set_params(classifier__max_depth=max_depth)

  scores = cross_val_score(dt_model, X, y, cv=cv_folds, scoring='accuracy')

  return scores.mean()

In [ ]:
study_decision_tree = optuna.create_study(direction='maximize')
study_decision_tree.optimize(decisiontree_optuna, n_trials=200)

In [ ]:
print(f'Melhor acurácia: {study_decision_tree.best_value}')
print(f'Melhores parâmetros: {study_decision_tree.best_params}')

### Visualizar Arvore

In [ ]:
X_train_tree = X.copy()
X_train_tree['localizacao_label'] = X_train_tree.localizacao.astype('category').cat.codes
X_train_tree['atividade_economica_label'] = X_train_tree.atividade_economica.astype('category').cat.codes
X_train_tree.drop(columns=['localizacao', 'atividade_economica'], axis=1, inplace=True)
X_train_tree.rename(columns={'localizacao_label': 'localizacao', 'atividade_economica': 'atividade_economica'}, inplace=True)
X_train_tree.head(10)

In [ ]:
clf_decisiontree = DecisionTreeClassifier(min_samples_leaf=study_decision_tree.best_params['min_samples_leaf'],
                                          max_depth=study_decision_tree.best_params['max_depth'])

y_train_tree = y.copy()
clf_decisiontree.fit(X_train_tree, y_train_tree)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), dpi=600)

plot_tree(clf_decisiontree,
          feature_names=X_train_tree.columns.to_numpy(),
          class_names=lista_segmento,
          filled=True)

## Salvar modelo

In [ ]:
import joblib

dt_model_tunado = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('classifier', DecisionTreeClassifier(min_samples_leaf=study_decision_tree.best_params['min_samples_leaf'],
                                                                      max_depth=study_decision_tree.best_params['max_depth']))])

dt_model_tunado.fit(X, y)

joblib.dump(dt_model_tunado, './modelo_classificacao_decision_tree.pkl')

### Entrega modelo

In [ ]:
import gradio as gr
modelo = joblib.load('./modelo_classificacao_decision_tree.pkl')

def predict(file):
  df_empresas = pd.read_csv(file.name)
  y_pred = modelo.predict(df_empresas)
  df_segmentos = pd.DataFrame(y_pred, columns=['segmento_de_clientes'])
  df_predicoes = pd.concat([df_empresas, df_segmentos], axis=1)
  df_predicoes.to_csv('./predicoes.csv', index=False)
  return './predicoes.csv'

demo = gr.Interface(
  predict,
  gr.File(file_types=['.csv']),
  "file"
)

demo.launch()